In [1]:
%pip install neo4j


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


URI = "bolt://localhost:7687"  # Use your actual Bolt URI
USERNAME = "neo4j"  # Default username
PASSWORD = "1122334455"  # Replace with your actual password

In [2]:
from neo4j import GraphDatabase

# Connection details
URI = "bolt://localhost:7687"  # Use your actual Bolt URI
USERNAME = "neo4j"  # Default username
PASSWORD = "1122334455"  # Replace with your actual password

# Connect to Neo4j
driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))

# Test connection function
def test_connection(tx):
    result = tx.run("RETURN 'Neo4j Connection Successful!' AS message")
    return result.single()["message"]

# Run the test query
with driver.session() as session:
    message = session.execute_read(test_connection)
    print("✅", message)

# Close the driver
driver.close()

✅ Neo4j Connection Successful!


In [3]:
# Load the cleaned JSON file
import json

# file_path = "cleaned_relationships.json"  # Update with the correct path
file_path = "layer2_cleaned_relationships.json"  # Update with the correct path

with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

FileNotFoundError: [Errno 2] No such file or directory: 'layer2_cleaned_relationships.json'

In [4]:
# Function to insert data into Neo4j
def insert_into_neo4j(tx, act_name, relationship, target_act):
    query = """
    MERGE (a:Act {name: $act_name})
    MERGE (b:Act {name: $target_act})
    MERGE (a)-[r:RELATES_TO {type: $relationship}]->(b)
    RETURN a, r, b
    """
    tx.run(query, act_name=act_name, relationship=relationship, target_act=target_act)

In [5]:
# Insert data into Neo4j
with driver.session() as session:
    for entry in data:
        act_name = "Family Violence Act 2018"  # Main Act affecting others
        target_act = entry["name"]
        relationship = entry["relationship"]

        session.write_transaction(insert_into_neo4j, act_name, relationship, target_act)

print("✅ Data successfully inserted into Neo4j!")

# Close the database connection
driver.close()

/var/folders/sj/y7cwpkns6vz2pv0mymm2qrkr0000gn/T/ipykernel_26296/519014984.py:2: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
/var/folders/sj/y7cwpkns6vz2pv0mymm2qrkr0000gn/T/ipykernel_26296/519014984.py:8: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(insert_into_neo4j, act_name, relationship, target_act)


✅ Data successfully inserted into Neo4j!


In [7]:
from neo4j import GraphDatabase
import json

# Neo4j Connection Details
URI = "bolt://localhost:7687"  # Use your actual Bolt URI
USERNAME = "neo4j"  # Default username
PASSWORD = "1122334455"  # Replace with your actual password

# Load Extracted Data
json_file = "cleaned_relationships.json"

with open(json_file, "r", encoding="utf-8") as file:
    extracted_data = json.load(file)

# Connect to Neo4j
driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))

# Function to insert data into Neo4j
def insert_relationship(tx, act_name, related_act, relationship_type):
    # Ensure the relationship type is one of the valid types
    valid_relationships = ["AMENDED", "REFERENCED", "REPEALED"]
    
    # Only insert valid relationship types
    if relationship_type.upper() in valid_relationships:
        query = f"""
        MERGE (a:Act {{name: $act_name}})
        MERGE (b:Act {{name: $related_act}})
        MERGE (a)-[r:{relationship_type.upper()}]->(b)
        RETURN a, b, r
        """
        tx.run(query, act_name=act_name, related_act=related_act)

# Delete old relationships before inserting new ones
with driver.session() as session:
    session.run("MATCH (a:Act)-[r]->(b:Act) DELETE r")
    print("✅ Deleted old relationships")

    # Insert new relationships
    for entry in extracted_data:
        act_name = "Family Violence Act 2018"  # Your main act
        related_act = entry["name"]
        relationship_type = entry["relationship"]  # Will be "Amended", "Referenced", or "Repealed"

        session.execute_write(insert_relationship, act_name, related_act, relationship_type)

print("✅ Inserted new relationships successfully!")

# Close Neo4j connection
driver.close()

✅ Deleted old relationships


KeyError: 'name'

In [1]:
import json
from neo4j import GraphDatabase

# Connection details
URI = "bolt://localhost:7687"
USERNAME = "neo4j"
PASSWORD = "1122334455"  # Use your actual Neo4j password

# Connect to Neo4j
driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))

# Test connection
def test_connection(tx):
    result = tx.run("RETURN 'Neo4j Connection Successful!' AS message")
    return result.single()["message"]

with driver.session() as session:
    message = session.execute_read(test_connection)
    print("✅", message)

# Load combined relationships file
with open("layer_2_relationships.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Function to insert a relationship
def insert_relationship(tx, source, target, relationship_type):
    valid_relationships = ["AMENDED", "REFERENCED", "REPEALED"]
    if relationship_type.upper() in valid_relationships:
        query = f"""
        MERGE (a:Act {{name: $source}})
        MERGE (b:Act {{name: $target}})
        MERGE (a)-[r:{relationship_type.upper()}]->(b)
        RETURN a, b, r
        """
        tx.run(query, source=source, target=target)

# Clear existing relationships
with driver.session() as session:
    session.run("MATCH (a:Act)-[r]->(b:Act) DELETE r")
    print("🧹 Deleted existing relationships.")

    # Insert new ones
    for entry in data:
        session.execute_write(
            insert_relationship,
            entry["source"],
            entry["target"],
            entry["relationship"]
        )

print("✅ Combined relationships inserted into Neo4j.")

# Close connection
driver.close()

Transaction failed and will be retried in 1.1066123638287784s (Couldn't connect to localhost:7687 (resolved to ('[::1]:7687', '127.0.0.1:7687')):
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [Errno 61] Connection refused)
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 61] Connection refused))
Transaction failed and will be retried in 1.7178643035674932s (Couldn't connect to localhost:7687 (resolved to ('[::1]:7687', '127.0.0.1:7687')):
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [Errno 61] Connection refused)
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 61] Connection refused))
Transaction failed and will be retried in 3.6732124185869854s (Couldn't connect to localhost:7687 (resolved to ('[::1]:7687', '127.0.0.1:7687')):
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [Errno 61] Connection

KeyboardInterrupt: 

In [2]:
# Load the cleaned JSON file
import json

file_path = "layer_2_relationships_cleaned.json"  # Update with the correct path

with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

In [4]:
# import json
# from neo4j import GraphDatabase

# # Connection details
# URI = "bolt://localhost:7687"
# USERNAME = "neo4j"
# PASSWORD = "1122334455"  # Use your actual Neo4j password

# # Connect to Neo4j
# driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))

# # Test connection
# def test_connection(tx):
#     result = tx.run("RETURN 'Neo4j Connection Successful!' AS message")
#     return result.single()["message"]

# with driver.session() as session:
#     message = session.execute_read(test_connection)
#     print("✅", message)

# # Load combined relationships file
# with open("layer_2_relationships_cleaned.json", "r", encoding="utf-8") as file:
#     data = json.load(file)

# # Function to insert a relationship
# def insert_relationship(tx, source, target, relationship_type):
#     valid_relationships = ["AMENDED", "REFERENCED", "REPEALED"]
#     if relationship_type.upper() in valid_relationships:
#         query = f"""
#         MERGE (a:Act {{name: $source}})
#         MERGE (b:Act {{name: $target}})
#         MERGE (a)-[r:{relationship_type.upper()}]->(b)
#         RETURN a, b, r
#         """
#         tx.run(query, source=source, target=target)

# # Clear existing relationships
# with driver.session() as session:
#     session.run("MATCH (a:Act)-[r]->(b:Act) DELETE r")
#     print("Deleted existing relationships.")

#     # Insert new ones
#     for entry in data:
#         session.execute_write(
#             insert_relationship,
#             entry["source"],
#             entry["target"],
#             entry["relationship"]
#         )

# print("✅ Combined relationships inserted into Neo4j.")

# # Close connection
# driver.close()

✅ Neo4j Connection Successful!
Deleted existing relationships.


AttributeError: 'NoneType' object has no attribute 'upper'

In [2]:
# import json
# from neo4j import GraphDatabase

# # Neo4j connection details
# URI = "bolt://localhost:7687"
# USERNAME = "neo4j"
# PASSWORD = "1122334455"  # Change this to your actual password

# # Connect to Neo4j
# driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))

# # Test the connection
# def test_connection(tx):
#     result = tx.run("RETURN 'Neo4j Connection Successful!' AS message")
#     return result.single()["message"]

# with driver.session() as session:
#     message = session.execute_read(test_connection)
#     print("✅", message)

# # Load cleaned relationship data
# with open("layer_2_relationships_cleaned.json", "r", encoding="utf-8") as file:
#     data = json.load(file)

# # Function to insert one relationship into Neo4j
# def insert_relationship(tx, source, target, relationship_type):
#     if not (source and target and relationship_type):
#         return  # Skip if any part is missing

#     relationship_type = str(relationship_type).upper()
#     valid_relationships = {"AMENDED", "REPEALED", "REFERENCED"}

#     if relationship_type in valid_relationships:
#         query = f"""
#         MERGE (a:Act {{name: $source}})
#         MERGE (b:Act {{name: $target}})
#         MERGE (a)-[r:{relationship_type}]->(b)
#         RETURN a, b, r
#         """
#         tx.run(query, source=source, target=target)

# # Step 1: Clear existing Act relationships
# with driver.session() as session:
#     session.run("MATCH (a:Act)-[r]->(b:Act) DELETE r")
#     print("🗑️ Deleted existing relationships.")

#     # Step 2: Insert new relationships
#     for entry in data:
#         source = entry.get("source")
#         target = entry.get("target")
#         relationship = entry.get("relationship")

#         if source and target and relationship:
#             session.execute_write(insert_relationship, source, target, relationship)
#         else:
#             print(f"⚠️ Skipped invalid entry: {entry}")

# print("✅ All valid relationships inserted into Neo4j.")

# # Close the Neo4j connection
# driver.close()

✅ Neo4j Connection Successful!
🗑️ Deleted existing relationships.
⚠️ Skipped invalid entry: {'source': 'Customs and Excise Act 2018', 'target': None, 'relationship': None}
⚠️ Skipped invalid entry: {'source': 'Child Support Amendment Act 2006', 'target': None, 'relationship': None}
⚠️ Skipped invalid entry: {'source': 'Child Support Act 1991', 'target': None, 'relationship': None}
⚠️ Skipped invalid entry: {'source': 'Mining Act 1971', 'target': None, 'relationship': 'Referenced'}
⚠️ Skipped invalid entry: {'source': 'Land Transfer Act 1952', 'target': None, 'relationship': 'Referenced'}
⚠️ Skipped invalid entry: {'source': 'Government Superannuation Fund Amendment Act 1995', 'target': None, 'relationship': 'Referenced'}
⚠️ Skipped invalid entry: {'source': 'Government Superannuation Fund Amendment Act 2001', 'target': None, 'relationship': 'Referenced'}
⚠️ Skipped invalid entry: {'source': 'Stamp Duty Abolition Act 1999', 'target': None, 'relationship': 'Referenced'}
⚠️ Skipped invali

In [3]:

import json
from neo4j import GraphDatabase

# Neo4j connection details
URI = "bolt://localhost:7687"
USERNAME = "neo4j"
PASSWORD = "1122334455"  # Change this to your actual password

# Connect to Neo4j
driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))

# Test the connection
def test_connection(tx):
    result = tx.run("RETURN 'Neo4j Connection Successful!' AS message")
    return result.single()["message"]

with driver.session() as session:
    message = session.execute_read(test_connection)
    print("✅", message)

# 🔄 Step 0: Clear the entire database
with driver.session() as session:
    session.run("MATCH (n) DETACH DELETE n")
    print("🧹 All nodes and relationships deleted.")

# Load cleaned relationship data
with open("layer_2_relationships_cleaned.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Function to insert one relationship into Neo4j
def insert_relationship(tx, source, target, relationship_type):
    if not (source and target and relationship_type):
        return  # Skip if any part is missing

    relationship_type = str(relationship_type).upper()
    valid_relationships = {"AMENDED", "REPEALED", "REFERENCED"}

    if relationship_type in valid_relationships:
        query = f"""
        MERGE (a:Act {{name: $source}})
        MERGE (b:Act {{name: $target}})
        MERGE (a)-[r:{relationship_type}]->(b)
        RETURN a, b, r
        """
        tx.run(query, source=source, target=target)

# Step 1: Insert new relationships
with driver.session() as session:
    for entry in data:
        source = entry.get("source")
        target = entry.get("target")
        relationship = entry.get("relationship")

        if source and target and relationship:
            session.execute_write(insert_relationship, source, target, relationship)
        else:
            print(f"⚠️ Skipped invalid entry: {entry}")

print("✅ All valid relationships inserted into Neo4j.")

# Close the Neo4j connection
driver.close()

✅ Neo4j Connection Successful!
🧹 All nodes and relationships deleted.
⚠️ Skipped invalid entry: {'source': 'Customs and Excise Act 2018', 'target': None, 'relationship': None}
⚠️ Skipped invalid entry: {'source': 'Child Support Amendment Act 2006', 'target': None, 'relationship': None}
⚠️ Skipped invalid entry: {'source': 'Child Support Act 1991', 'target': None, 'relationship': None}
⚠️ Skipped invalid entry: {'source': 'Mining Act 1971', 'target': None, 'relationship': 'Referenced'}
⚠️ Skipped invalid entry: {'source': 'Land Transfer Act 1952', 'target': None, 'relationship': 'Referenced'}
⚠️ Skipped invalid entry: {'source': 'Government Superannuation Fund Amendment Act 1995', 'target': None, 'relationship': 'Referenced'}
⚠️ Skipped invalid entry: {'source': 'Government Superannuation Fund Amendment Act 2001', 'target': None, 'relationship': 'Referenced'}
⚠️ Skipped invalid entry: {'source': 'Stamp Duty Abolition Act 1999', 'target': None, 'relationship': 'Referenced'}
⚠️ Skipped in

MATCH (a)-[r]->(b) RETURN a, r, b